# Collecte d’avis clients via une API web (cas Decathlon)

**Objectif du TP.** Mettre en œuvre une collecte d’avis produits à partir d’une **API JSON** identifiée sur un site e-commerce, puis transformer la réponse en un tableau analysable.

**Pourquoi l’API plutôt que le scraping HTML ?**
- L’API renvoie des **données structurées** (JSON) → plus simple et robuste que parcourir du HTML.
- Même source de données pour différentes interfaces (site, app) → **cohérence** et **fraîcheur**.
- Meilleure **reproductibilité** pour les analyses marketing (moins sensible aux changements d’interface).

**Rappel** : respecter les CGU, vérifier *robots.txt*, limiter la charge des requêtes, anonymiser si besoin.

## Test minimal : interroger l’API et inspecter la réponse

**Ce que fait le code ci-dessous :**
- Envoie une **requête HTTP GET** vers l’endpoint d’avis du produit `8584565`.
- Fournit des **headers** simples (`accept`, `user-agent`, `referer`) pour mimer un navigateur standard.
- Affiche le **statut HTTP** (attendu : `200 OK` si l’API répond).  
- Tente de décoder la **réponse en JSON** et l’imprime avec un retrait (`indent=2`) pour lecture humaine.  
- Tronque l’affichage aux ~4000 premiers caractères pour garder l’aperçu lisible.

**À quoi s’attendre ?**
- Une structure de type :  
  ```json
  {
    "reviews": [
      { "id": "...", "rating": {"code": 5, "label": "Excellent"}, "title": "...", "comment": "...", ... },
      ...
    ]
  }


In [8]:
# pip install requests
import requests, json

url = "https://www.decathlon.fr/fr/ajax/nfs/openvoice/reviews/product/8584565?range=0-9"
headers = {
    "accept": "application/json, text/plain, */*",
    "user-agent": "Mozilla/5.0",
    "referer": "https://www.decathlon.fr/",
}

r = requests.get(url, headers=headers, timeout=20)
print("status:", r.status_code)
# Si c'est bien du JSON, on l'affiche joliment
try:
    data = r.json()
    print(json.dumps(data, ensure_ascii=False, indent=2)[:4000])  # on tronque pour lire confortablement
except ValueError:
    print("Réponse non-JSON, voici le début:")
    print(r.text[:1000])


status: 200
{
  "reviews": [
    {
      "id": "a61515c8-8b2d-4e67-988c-072283e34b49",
      "rating": {
        "code": 5,
        "label": "Excellent"
      },
      "title": "Super et spacieuse",
      "comment": "Super et spacieuse",
      "status": "published",
      "provider": "decathlon",
      "publisherDate": "2025-08-27T20:00:26+00:00",
      "recommended": true,
      "attributes": [
        {
          "attribute": "58",
          "rating": 5,
          "label": "habitabilité"
        },
        {
          "attribute": "44",
          "rating": 4,
          "label": "facilité de montage / démontage"
        },
        {
          "attribute": "335",
          "rating": 5,
          "label": "réduction de la chaleur"
        },
        {
          "attribute": "price_quality",
          "rating": 3,
          "label": "Rapport qualité/prix"
        },
        {
          "attribute": "look_design",
          "rating": 4,
          "label": "Look / Design"
        }
      ]

## Lecture de la structure JSON renvoyée

**Observations principales (extrait) :**
- `reviews` : **liste** d’avis (un dictionnaire par avis).
- `id` : identifiant unique de l’avis.
- `rating` : objet imbriqué → `code` (valeur numérique de la note), `label` (ex. *Excellent*).
- `title`, `comment` : titre court et texte de l’avis (le commentaire peut contenir des balises HTML simples, ex. `<br>`).
- `author.username` : pseudonyme de l’auteur.
- `publisherDate` : date ISO 8601 (UTC) de publication.
- `tags.buyer` : indicateur **Achat confirmé** (fiabilité perçue).
- `attributes` : liste d’attributs notés (ex. *habitabilité*, *facilité de montage*, etc.).
- `locale`, `country.code` : langue/zone (ex. `fr_FR`, `FR`).

**Conséquence pour l’analyse :**  
il est pertinent d’**aplatir** (flatter) la structure pour obtenir un **tableau** : une ligne par avis, des colonnes pour les champs clés.


## Transformation en tableau : du JSON imbriqué au DataFrame

**Objectif :** construire un **DataFrame** avec les colonnes pertinentes :
- Identifiants et métadonnées : `review_id`, `publisherDate`, `locale`, `country`.
- Contenu et évaluation : `rating` (code numérique), `rating_label`, `title`, `comment`, `author`.
- Confiance : `buyer_confirmed` (achat confirmé).
- Attributs d’usage (facultatif) : extraire certaines dimensions notées par les clients  
  (ex. `habitabilité`, `facilité de montage / démontage`, `réduction de la chaleur`, `Rapport qualité/prix`, `Look / Design`).

**Choix de modélisation :**
- Les `attributes` sont une **liste** → on les convertit en colonnes (ex. `Habitabilité`, `Montage/Démontage`, etc.).  
- Les valeurs manquantes sont possibles (tous les avis ne renseignent pas chaque attribut) → prévoir `NaN`.

**Résultat attendu :** un tableau propre, immédiatement exploitable en marketing (tri, filtrage, agrégations, graphiques).


## On récupère tous les avis de la page en utilisant le PRODUCT_ID de l'URL

In [9]:
# pip install requests pandas
import requests, pandas as pd, json

PRODUCT_ID = "8584565"
URL = f"https://www.decathlon.fr/fr/ajax/nfs/openvoice/reviews/product/{PRODUCT_ID}?range=0-9"

headers = {
    "accept": "application/json, text/plain, */*",
    "user-agent": "Mozilla/5.0",
    "referer": "https://www.decathlon.fr/",
}

r = requests.get(URL, headers=headers, timeout=20)
r.raise_for_status()
data = r.json()

# ——— mapping des champs utiles
rows = []
for rev in data.get("reviews", []):
    # aplatir les attributes (liste) -> dict {label: rating}
    attr = {}
    for a in rev.get("attributes", []):
        label = (a.get("label") or "").strip()
        if label:
            attr[label] = a.get("rating")
    rows.append({
        "review_id": rev.get("id"),
        "rating": (rev.get("rating") or {}).get("code"),
        "rating_label": (rev.get("rating") or {}).get("label"),
        "title": rev.get("title"),
        "comment": rev.get("comment"),
        "author": (rev.get("author") or {}).get("username"),
        "publisherDate": rev.get("publisherDate"),
        "recommended": rev.get("recommended"),
        "buyer_confirmed": (rev.get("tags") or {}).get("buyer"),
        "locale": rev.get("locale"),
        "country": (rev.get("country") or {}).get("code"),
        # attributs utiles (exemples)
        "Habitabilité": attr.get("habitabilité"),
        "Montage/Démontage": attr.get("facilité de montage / démontage"),
        "Réduction chaleur": attr.get("réduction de la chaleur"),
        "Rapport Q/P": attr.get("Rapport qualité/prix"),
        "Look/Design": attr.get("Look / Design"),
    })

df = pd.DataFrame(rows)
print(df.head(3))
df.to_csv("decathlon_reviews_page1.csv", index=False)
with open("decathlon_reviews_page1.json", "w", encoding="utf-8") as f:
    json.dump(rows, f, ensure_ascii=False, indent=2)

print(f"\n{len(df)} avis sur cette page. Fichiers écrits : decathlon_reviews_page1.csv / .json")


                              review_id  rating rating_label  \
0  a61515c8-8b2d-4e67-988c-072283e34b49       5    Excellent   
1  54cdfc90-862a-4e3d-acc0-d9add7d82d72       5    Excellent   
2  0207fa4c-2aba-4a49-8974-8504627f1b02       2     Passable   

                               title  \
0                 Super et spacieuse   
1  Utilisée depuis 3 ans pour les...   
2  Vraiment déçu de cet achat.Pos...   

                                             comment    author  \
0                                 Super et spacieuse    Maxime   
1  Utilisée depuis 3 ans pour les vacances, cette...  caroline   
2  Vraiment déçu de cet achat.<br>Possédant l'anc...  Jonathan   

               publisherDate recommended  buyer_confirmed locale country  \
0  2025-08-27T20:00:26+00:00        True             True  fr_FR      FR   
1  2025-08-27T12:01:01+00:00        True            False  fr_FR      FR   
2  2025-08-26T14:35:41+00:00       False             True  fr_FR      FR   

   Habitabil

## Lecture rapide du résultat

- Vérifier l’alignement des colonnes (valeurs attendues vs. *NaN*).  
- Contrôler la **cohérence** : 
  - `rating` ∈ {1, 2, 3, 4, 5} ; 
  - `publisherDate` au format ISO ; 
  - `buyer_confirmed` ∈ {True/False}.  
- Noter la présence éventuelle de **balises HTML** dans `comment` → un nettoyage peut être nécessaire pour l’analyse textuelle.

**Interprétation marketing immédiate (sur cet échantillon de 10 avis) :**
- Distribution des notes (tendance globale de satisfaction).  
- Fréquence des **achats confirmés** (indicateur de fiabilité perçue).  
- Attributs souvent bien notés / plus critiques (ex. *Rapport Q/P*).


## On affiche le dataframe

In [10]:
df

,review_id,rating,rating_label,title,comment,author,publisherDate,recommended,buyer_confirmed,locale,country,Habitabilité,Montage/Démontage,Réduction chaleur,Rapport Q/P,Look/Design
0,a61515c8-8b2d-4e67-988c-072283e34b49,5,Excellent,Super et spacieuse,Super et spacieuse,Maxime,2025-08-27T20:00:26+00:00,True,True,fr_FR,FR,5,4,5,3,4
1,54cdfc90-862a-4e3d-acc0-d9add7d82d72,5,Excellent,Utilisée depuis 3 ans pour les...,"Utilisée depuis 3 ans pour les vacances, cette...",caroline,2025-08-27T12:01:01+00:00,True,False,fr_FR,FR,5,5,5,4,4
2,0207fa4c-2aba-4a49-8974-8504627f1b02,2,Passable,Vraiment déçu de cet achat.Pos...,Vraiment déçu de cet achat.<br>Possédant l'anc...,Jonathan,2025-08-26T14:35:41+00:00,False,True,fr_FR,FR,4,4,1,1,4
3,05b3cb64-a7aa-4c83-9079-5d3f7472583f,5,Excellent,"Excellente tente, facile à ins...","Excellente tente, facile à installer grâce au ...",Jay,2025-08-26T11:41:59+00:00,True,True,fr_FR,FR,5,5,4,5,4
4,10aec241-0dfe-492f-bce8-565040c0df14,5,Excellent,Très gros volume et facilité d...,Très gros volume et facilité de montage et de ...,SYLVAIN,2025-08-25T18:30:00+00:00,True,True,fr_FR,FR,5,5,4,5,4
5,a7508b26-4ce9-4666-ba0b-c036538811cc,5,Excellent,"Très grande ,très pratique et ...","Très grande ,très pratique et se gonfle rapide...",Céline,2025-08-25T07:12:01+00:00,True,True,fr_FR,FR,5,5,5,5,5
6,3d5f2373-3820-4149-a909-864ab3cb8388,5,Excellent,"Très bon produit, facile à mon...","Très bon produit, facile à monter et démonter",Jean-Francois,2025-08-21T06:09:02+00:00,True,True,fr_FR,FR,5,5,5,5,5
7,2758978b-501f-4b9f-8d14-e5009a04404a,4,Bon,Passage pour nous à l attente ...,Passage pour nous à l attente gonflable . Nous...,Stephanie,2025-08-20T09:34:41+00:00,None,True,fr_FR,FR,5,5,5,5,5
8,f59d4bba-a43c-4083-921a-38a827c362a9,5,Excellent,Montage super rapide et très f...,Montage super rapide et très facile <br>,Nathalie,2025-08-19T14:01:05+00:00,True,True,fr_FR,FR,5,5,5,5,5
9,f7dbc053-d218-4236-8bc9-9d071e5d0e4a,5,Excellent,Top pour les vacances,Top pour les vacances,Virginie,2025-08-18T18:30:07+00:00,None,True,fr_FR,FR,5,5,4,4,4
